# Neighborhood Modeling

TBD

# Summary

TBD

In [4]:
# Standard imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import geopandas as gpd

# Set style
plt.style.use('seaborn')

In [5]:
# Import util file

from util import dickey_fuller, process_neighborhood, report_metrics, plot_results

In [6]:
# Modeling

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.graphics.tsaplots as tsa

import pmdarima as pm
from pmdarima import model_selection
from pmdarima.utils import decomposed_plot
from pmdarima.arima import decompose
from pmdarima.arima.stationarity import ADFTest

from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [18]:
df_citibike = pd.read_csv('./clean_data/final_clean_ridership.csv', low_memory=False)

In [20]:
# Import clean dataset

df_citibike = pd.read_csv('./clean_data/final_clean_ridership.csv', low_memory=False)

# Convert starttime to date and set as index

df_citibike['starttime'] = pd.to_datetime(df['starttime'])

df_citibike.set_index('starttime', inplace=True)

# Create the daily file

daily_ridership = df_citibike[['ride_count']].resample('1D').sum()

# Filter for only data prior to 5/1/2021 (logic for this is in the EDA notebook)
# daily_ridership = daily_ridership[daily_ridership.index < '2021-06-01']

# Backfill dates with 0 rides (logic for this change in EDA notebook)
daily_ridership['ride_count'] = daily_ridership['ride_count'].replace(to_replace=0, method='bfill')

In [21]:
# Read in clean neighborhood list and convert to list

temp_df = pd.read_csv('./clean_data/neighborhood_list.csv', low_memory=False)

neighborhood_list = list(temp_df.iloc[:,1])

In [28]:
# Process each of the DFs in the neighborhood list

df_dict = {}

for neighborhood in neighborhood_list:
    df = process_neighborhood(df_citibike, neighborhood)
    df_dict[neighborhood] = df

In [31]:
# Resample for weekly based on the results of the overall modeling

for neighborhood, df in df_dict.items():
    df_dict[neighborhood] = df[['ride_count']].resample('W-MON').sum()